# VOQC  <img src="umd.jpeg" alt="Drawing" style="width: 100px" align="right">

</style>
Welcome to VOQC, a verfied optimizer for quantum circuits! This tutorial is 

In [9]:
from qiskit import QuantumCircuit
from interop.voqc import SQIR
from interop.qiskit.voqc_optimization import VOQC
from qiskit.transpiler import PassManager
from qiskit.qasm import pi

## Passing a Qiskit Circuit Object to VOQC

Because of VOQC's interoperability with Cirq and Qiskit, it is now possible to pass a Qiskit circuit object through the VOQC optimizations and receive an optimized Qiskit circuit. 

The VOQC transpiler takes an input in the form of a Directed Acyclic Graph (like the built-in Qiskit optimizations), converts this into a QASM string, passes the string to VOQC, optimizes the circuit, and returns the optimized QASM string to convert back to a DAG. There is an optional list argument for the VOQC compiler pass, which will be explained in the *Running Individual Optimizations* section. 

Given a sample circuit **"circ"** and Pass Manager **"pm"**, you can append the VOQC transpiler pass to **"pm"** without an argument, **VOQC()**. This transpiler pass will run the primary optimization function in VOQC, a specific order of the five unitary optimizations listed in the introduction. 

A basic example of this is shown below. More detail about each unitary optimization will be provided later in the tutorial. 


## Running Individual Unitary Optimizations

While appending **VOQC()** to the pass manager runs the primary VOQC optimization function, there are scenarios where you may want to run one or more of the unitary optimizations of VOQC. These five optimizations are *not propagation*, *cancel single qubit gates*, *cancel two qubit gates*, *merge rotations*, and *hadamard reduction*. In this section, we will describe each of the five aforementioned optimizations, including their purpose and how to incorprate them into your compiler pass.

As always, the implementations of these optimizations can be found <a href="https://github.com/inQWIRE/SQIR">here</a> and the paper itself can be found <a href="https://arxiv.org/abs/1912.02250">here</a>. 

### Not Propagation

<p>The "not propagation" optimization cancels two NOT gates (X) that are either adjacent or separated by a circuit that commutes with X. For commuting circuits, the identities below are applied repeatedly to simplify the circuit.</p> 

<p><center>$X q;  H q \equiv H q;  Z q$</center></p>
<p><center>$X q;  Rz(k) q \approx Rz(2-k) q;  X q$</center></p>
<p><center>$X q_{1};  CNOT q_{1} q_{2} \equiv CNOT q_{1} q_{2};  X q_{1};  X q_{2}$</center></p>
<p><center>$X q_{2};  CNOT q_{1} q_{2} \equiv CNOT q_{1} q_{2};  X q_{2}$</center></p>




<p>In the example below, the leftmost X gate propagates through the CNOT gate, invoking the fourth identity above. Now, the circuit consists of a CNOT gate and two adjacent NOT gates. These adjacent X gates are "cancelled," leading to the final circuit, a single CNOT gate. </p>

<p>To use this optimization, append "not_propagation" to the optional list argument in the VOQC compiler pass. 


In [15]:
#Build Circuit with Two Qubits and Two Gates (CNOT and NOT)
circ = QuantumCircuit(2)
circ.x(1)
circ.cx(0, 1)
circ.x(1)
print("Before Optimization:")
print(circ)

#Append "not_propagation" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["not_propagation"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

Before Optimization:
                    
q_0: ───────■───────
     ┌───┐┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├┤ X ├
     └───┘└───┘└───┘
After Optimization:
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘


### Cancel Single Qubit Gates

In [10]:
#Build Circuit with One Qubit and Three Gates (Rz and 2 Hadamard)
circ = QuantumCircuit(1)
circ.rz(pi/2, 0)
circ.h(0)
circ.h(0)
print("Before Optimization:")
print(circ)

#Append "cancel_single_qubit_gates" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["cancel_single_qubit_gates"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

Before Optimization:
     ┌──────────┐┌───┐┌───┐
q_0: ┤ RZ(pi/2) ├┤ H ├┤ H ├
     └──────────┘└───┘└───┘
After Optimization:
     ┌───┐
q_0: ┤ S ├
     └───┘


### Cancel Two Qubit Gates

In [11]:
#Build Circuit with Two Qubits and Three Gates (2 CNOT and Hadamard)
circ = QuantumCircuit(2)
circ.cx(0, 1)
circ.cx(0, 1)
circ.h(0)
print("Before Optimization:")
print(circ)

#Append "cancel_two_qubit_gates" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["cancel_two_qubit_gates"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

Before Optimization:
               ┌───┐
q_0: ──■────■──┤ H ├
     ┌─┴─┐┌─┴─┐└───┘
q_1: ┤ X ├┤ X ├─────
     └───┘└───┘     
After Optimization:
     ┌───┐
q_0: ┤ H ├
     └───┘
q_1: ─────
          


### Merge Rotations

In [13]:
#Build Circuit with One Qubit and Two Gates (Two Rz)
circ = QuantumCircuit(1)
circ.rz(pi/4, 0)
circ.rz(pi/4, 0)
print("Before Optimization:")
print(circ)

#Append "merge_rotations" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["merge_rotations"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

Before Optimization:
     ┌──────────┐┌──────────┐
q_0: ┤ RZ(pi/4) ├┤ RZ(pi/4) ├
     └──────────┘└──────────┘
After Optimization:
     ┌───┐
q_0: ┤ S ├
     └───┘


### Hadamard Reduction

In [14]:
#Build Circuit with Two Qubits and Two Gates (CNOT and NOT)
circ = QuantumCircuit(1)
circ.h(0)
circ.rz(pi/2, 0)
circ.h(0)

print("Before Optimization:")
print(circ)

#Append "hadamard_reduction" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["hadamard_reduction"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

Before Optimization:
     ┌───┐┌──────────┐┌───┐
q_0: ┤ H ├┤ RZ(pi/2) ├┤ H ├
     └───┘└──────────┘└───┘
After Optimization:
     ┌─────┐┌───┐┌─────┐
q_0: ┤ SDG ├┤ H ├┤ SDG ├
     └─────┘└───┘└─────┘
